In [1]:
!wget https://ftp.ensembl.org/pub/release-114/fasta/mus_musculus/cdna/Mus_musculus.GRCm39.cdna.all.fa.gz
!gunzip Mus_musculus.GRCm39.cdna.all.fa.gz
!wget https://ftp.ensembl.org/pub/release-114/fasta/mus_musculus/ncrna/Mus_musculus.GRCm39.ncrna.fa.gz
!gunzip Mus_musculus.GRCm39.ncrna.fa.gz

--2026-01-14 14:00:01--  https://ftp.ensembl.org/pub/release-114/fasta/mus_musculus/cdna/Mus_musculus.GRCm39.cdna.all.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50893631 (49M) [application/x-gzip]
Saving to: ‘Mus_musculus.GRCm39.cdna.all.fa.gz’

Mus_musculus.GRCm39 100%[===================>]  48.54M  11.3MB/s    in 4.8s    

2026-01-14 14:00:06 (10.0 MB/s) - ‘Mus_musculus.GRCm39.cdna.all.fa.gz’ saved [50893631/50893631]

--2026-01-14 14:00:07--  https://ftp.ensembl.org/pub/release-114/fasta/mus_musculus/ncrna/Mus_musculus.GRCm39.ncrna.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28414607 (27M) [application/x-gzip]
Saving to: ‘Mus_musculus.GRCm39.ncrna.fa.gz’

Mus_muscul

In [2]:
from baitshop import code_design, barcode_assignment
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
import Bio.SeqIO as SeqIO

In [3]:
average_expression = pd.read_csv('data/Mean_expression_per_cluster.csv', index_col=0)

In [4]:
# Identify genes with >4000 counts in any cluster (genes are indices)
high_expr_genes = average_expression[average_expression.max(axis=1) > 4000].index.tolist()
high_expr_genes

[]

In [5]:
# Download a 16 bit LJCR code with k=4 and t=3
code_design.download_ljcr_csv(v=16, k=4, t=3, output_path='ljcr_139_hd4.csv')

Saved 139 codewords to ljcr_139_hd4.csv


In [6]:
# Filter codewords with Hamming distance < 4
codewords = code_design.filter_hd_gt4('ljcr_139_hd4.csv')
len(codewords)

139

In [7]:
codeword_array = np.array([list(map(int, bc)) for bc in codewords])

In [8]:
codeword_list = [list(map(int, bc)) for bc in codewords]

In [9]:
# Create codeword_list as list of strings
codeword_strings = [''.join(map(str, bc)) for bc in codeword_array]
len(codeword_strings)

139

In [10]:
# Get hamming distances between codewords
hamming_condensed = pdist(codeword_array, metric='hamming')
hamming_matrix = squareform(hamming_condensed) * codeword_array.shape[1]
hamming_matrix

array([[0., 4., 4., ..., 8., 8., 8.],
       [4., 0., 4., ..., 8., 8., 8.],
       [4., 4., 0., ..., 8., 8., 8.],
       ...,
       [8., 8., 8., ..., 0., 4., 4.],
       [8., 8., 8., ..., 4., 0., 4.],
       [8., 8., 8., ..., 4., 4., 0.]])

In [11]:
# Get bulk RNA-seq isoform expression data, APPRIS data, and gene names
isoform_expression = pd.read_csv('data/Zhang_2023_Salmon_TPM.csv')
appris_data = pd.read_table('data/appris_data.principal.txt', sep='\t', header=None)
with open('data/gene_names.txt') as f:
    gene_names = f.read().splitlines()

In [12]:
# Rename columns for clarity
isoform_expression.columns = ['Name', 'TPM']

In [13]:
# Rename columns and remove '3'
appris_data.drop(columns=[3], inplace=True)
appris_data.columns = ['gene','gene_id', 'isoform', 'annotation_level']
appris_data

,gene,gene_id,isoform,annotation_level
0,Sf3b1,ENSMUSG00000025982,ENSMUST00000027127,PRINCIPAL:1
1,Fev,ENSMUSG00000055197,ENSMUST00000068631,PRINCIPAL:1
2,Septin2,ENSMUSG00000026276,ENSMUST00000027495,PRINCIPAL:1
3,Atf6,ENSMUSG00000026663,ENSMUST00000027974,PRINCIPAL:1
4,Nck2,ENSMUSG00000066877,ENSMUST00000086421,PRINCIPAL:1
...,...,...,...,...
33950,mt-Nd4l,ENSMUSG00000065947,ENSMUST00000084013,PRINCIPAL:1
33951,mt-Nd5,ENSMUSG00000064367,ENSMUST00000082418,PRINCIPAL:1
33952,mt-Nd4,ENSMUSG00000064363,ENSMUST00000082414,PRINCIPAL:1
33953,mt-Nd6,ENSMUSG00000064368,ENSMUST00000082419,PRINCIPAL:1


In [14]:
# Get transcript lengths from cDNA FASTA file
# Isoform is record.id, gene is record.description gene_symbol:"gene"
data = []
for record in SeqIO.parse('Mus_musculus.GRCm39.cdna.all.fa', 'fasta'):
	isoform = record.id.split('|')[0]
	length = len(record.seq)
	# Extract gene symbol from description
	if 'gene_symbol:' in record.description:
		gene = record.description.split('gene_symbol:')[1].split()[0]
	else:
		gene = None
	data.append({'gene': gene, 'isoform': isoform, 'length': length})

transcript_lengths = pd.DataFrame(data)
transcript_lengths

,gene,isoform,length
0,Igkv1-133,ENSMUST00000103304.3,374
1,Igkv1-132,ENSMUST00000103305.2,362
2,Igkv1-131,ENSMUST00000103306.2,362
3,Igkv17-127,ENSMUST00000200586.2,359
4,Igkv17-127,ENSMUST00000103309.3,341
...,...,...,...
116308,Gm14415,ENSMUST00000118260.2,1578
116309,Gm14404,ENSMUST00000119117.2,1660
116310,Gm16357,ENSMUST00000144091.2,474
116311,Gm13339,ENSMUST00000120651.2,510


In [15]:
# Make sure every gene in gene_names is in transcript_lengths
missing_genes = [gene for gene in gene_names if gene not in transcript_lengths['gene'].values]
missing_genes

[]

In [16]:
# Select the best isoform for each gene
best_isoforms = code_design.select_best_isoform(isoform_expression, gene_names, appris_data, transcript_lengths)

In [17]:
# Load Spearman correlation matrix
spearman_mat = np.load('data/Gene_Spearman.npy')

In [18]:
len(spearman_mat[0])

100

In [19]:
# Assign barcodes using low-rank Gromov-Wasserstein method
barcode_assignments = barcode_assignment.assign_barcodes_lowrank_gw(genes=gene_names, barcodes=codeword_strings,
                                                                    correlation_matrix=spearman_mat, distance_matrix=hamming_matrix,epsilon=1e-05,
                                                                    max_iterations=1000, variance_fraction=1)

In [20]:
# Evaluate the barcode assignment
eval = barcode_assignment.evaluate_assignment(correlation_matrix=spearman_mat,
                                             distance_matrix=hamming_matrix,
                                             assignment=barcode_assignments)
eval

{'mean_distance_high_corr': 6.357142857142857,
 'mean_distance_med_corr': 5.972754793138244,
 'mean_distance_low_corr': 5.987074829931973,
 'min_distance_high_corr': 4.0,
 'correlation_distance_pearson': 0.009832522845345677,
 'correlation_distance_spearman': 0.003816516813200746}

In [21]:
# Create dict of readout names to sequences
readouts = SeqIO.parse('data/69_bit_readouts.fasta', 'fasta')
readout_dict = {record.id: str(record.seq) for record in readouts}

In [22]:
readout_dict.keys()

dict_keys(['RS0015', 'RS0083', 'RS0095', 'RS0109', 'RS0175', 'RS0237', 'RS0247', 'RS0255', 'RS0307', 'RS0332', 'RS0343', 'RS0384', 'RS0406', 'RS0451', 'RS0468', 'RS0548', 'RS0578', 'RS0584', 'RS0639', 'RS0707', 'RS0708', 'RS0730', 'RS0763', 'RS0793', 'RS0805', 'RS0896', 'RS0820', 'RS0994', 'RS0996', 'RS0967', 'RS1040', 'RS1047', 'RS1199', 'RS1316', 'RS1312', 'RS1261', 'RS1326', 'RS1404', 'RS1334', 'RS3505', 'RS1470', 'RS1520', 'RS3583', 'RS1678', 'RS3592', 'RS3602', 'RS1982', 'RS3791', 'RS3811', 'RS3918', 'RS3544', 'RS4013', 'RS4027', 'RS4070', 'RS4098', 'RS4111', 'RS4140', 'RS4164', 'RS4194', 'RS4201', 'RS4216', 'RS4324', 'RS4341', 'RS4351', 'RS4363', 'RS4365', 'RS4465', 'RS4483', 'RS4554'])

In [23]:
# Create the codebook
codebook = code_design.create_codebook(genes=gene_names, barcodes=codeword_strings, assignment=barcode_assignments,
                                       isoform_df=best_isoforms, readouts=readout_dict)

In [24]:
codebook

,name,id,RS0015,RS0083,RS0095,RS0109,RS0175,RS0237,RS0247,RS0255,RS0307,RS0332,RS0343,RS0384,RS0406,RS0451,RS0468,RS0548
0,Adarb2,ENSMUST00000135574.8,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1
1,Erbb4,ENSMUST00000121473.8,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0
2,Trpm3,ENSMUST00000237357.2,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,Meis2,ENSMUST00000074285.8,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0
4,Nrxn3,ENSMUST00000163134.8,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Grm5,ENSMUST00000125009.9,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0
96,Gria1,ENSMUST00000036315.16,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
97,Arpp21,ENSMUST00000161467.8,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0
98,Chrm3,ENSMUST00000187510.7,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
